# 2-2 extract api list from java code

- step1 java code 코드 수집
- step2 api에서 정규식을 적용하여 전체 리스트 및 갯수 확인
- step3 api에서 비번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인
- step4 10000개의 리스트 중에서 영향도 있는 리스트만 각출
- step5 각 apk 파일들의 api 수집 및 정리


## step1 java code 코드 수집

In [1]:
import os

dir_names = ["Challenge_andro_2nd_dataset"]
dir_list = list()

i = 0
print( ".",end =' ')
for dir_name in dir_names:
    dir_name = dir_name + "_apktools"
            
    for root, dirs, files in os.walk(dir_name):
        for f in files:
            if f.endswith(".java"):
                dir_list.append(root + '/' + f )
                i+=1
                if i%10000 == 0:
                    print( 'Count: {}                 \r'.format(i),end ='')
                    
print( 'Count: {}                 \r'.format(i),end ='')

In [2]:
dir_list[:10]

['Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/g.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/e.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/h.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/d.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/f.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/c.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/a.java',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/out/com/a/b.java',
 'Challenge_andr

In [3]:
import pandas as pd

df = pd.read_csv('java_fuction_list.csv').T.drop('Unnamed: 0')
function_list = list(df.index)
function_list[:10]

FileNotFoundError: File b'java_fuction_list.csv' does not exist

## step5 각 apk 파일들의 4gram 수집 및 정리
apk_word_list
파일로부터 데이터를 불러와서 해시 값에 따라 counter 값 추가

In [ ]:
import re
from multiprocessing import Pool
from collections import Counter
import numpy as np

# Create three Counter objects to store positive, negative and total counts
word_function_list = Counter()

def file2strings(key):
    with open(key, encoding = "ISO-8859-1") as f:
        data = f.read()
        data = data.lower()
    
    word_count = Counter()
    filename = key.split("/")[1] # 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$FbLoginDialogListener.smali'
    
    pat = re.compile("([a-zA-Z0-9_]+)\(.*\)")
    for word in pat.findall(data):
        if word in function_list:
            word_count[word] += 1
            
    return filename, dict(word_count.most_common())

with Pool(16) as p:
    count = len(dir_list)
    print ("start")
    i = 0
    for filename, word_count in p.imap(file2strings, dir_list):
        word_function_list[filename] = word_count
                    
        i += 1    
        if i%100 == 0:
            print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
    

In [ ]:
len(total_counts)

In [ ]:
df1 = pd.DataFrame(word_function_list, index = function_list).T.fillna(0)
df1

In [ ]:
df1.sum()

In [ ]:
df1.to_csv('2nd_java_function.csv')